In [12]:
#Debjyoti's modifivation - V2

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, transpile
from qiskit.circuit.library import CDKMRippleCarryAdder 

from qiskit.providers.basic_provider import BasicSimulator
backend = BasicSimulator()



# plain_text = "a"
# printing original string 
# print("The original string is : " + str(plain_text))
 
ENCRYPTED_TEXT=''

KEY="101" #5 in decimal
n_k=len(KEY)
print("key=",int(KEY, 2))

# Defining Quantum registers
# for i in plain_text:
# using join() + ord() + format()


# res = ''.join(format(ord(plain_text), '07b')) # converting plain text to binary
# PLAIN_TEXT=str(res)

PLAIN_TEXT='11000'
print(PLAIN_TEXT)

n_p=len(PLAIN_TEXT)

adder = CDKMRippleCarryAdder(n_p, kind='full', name='Full Adder') # kind='full' indicates full adder
operand1 = QuantumRegister(n_p, name='key')
operand2 = QuantumRegister(n_p, name='plain_text')
anc = QuantumRegister(2, name='a') # to store carry in and carry out
cr = ClassicalRegister(n_p+1,name='cr')

circ = QuantumCircuit(operand1, operand2, anc, cr)

for i in range(len(KEY)):
    if KEY[i]=='1':
        circ.x(operand1[len(KEY)-1-i])

for i in range(len(PLAIN_TEXT)):
    if PLAIN_TEXT[i]=='1':
        circ.x(operand2[len(PLAIN_TEXT)-1-i])

# Note 1: CDKMRippleCarryAdder expects the qubits to have the order: carry in - 1st operand - 2nd operand - carry out. (changes will be on 2nd operator)
# Note 2: Python uses '+' operator to combine lists:

circ.append(adder, [anc[0]] + operand1[:] + operand2[:] + [anc[1]])
circ.measure(operand2[:] + [anc[1]], cr) #changes will be on operand 2 in other word (here) plain_text is changed 

# to change the key : take care of ordering the 1st operand and 2nd operand
# circ.append(adder, [anc[0]] + operand2[:] + operand1[:] + [anc[1]])
# circ.measure(operand1[:] + [anc[1]], cr) 

# print(circ)
# circ.draw("mpl")

tr_circ = transpile(circ, basis_gates = ['u3', 'cx'], optimization_level = 3)
result = backend.run(tr_circ).result()
counts = result.get_counts()

# print("encrypted text in binary:",list(counts.keys())[0])
encrypted_text = list(counts.keys())[0][1:]
print(encrypted_text)
chunks = [encrypted_text[i:i+8] for i in range(0, len(encrypted_text), 8)]

# Convert each chunk to integer and then to character
# resulting_char = ''.join([chr(int(chunk, 2)) for chunk in chunks])

# print("encrypted char",resulting_char)
# ENCRYPTED_TEXT+=resulting_char
    
# print("ENCRYPTED_TEXT :",ENCRYPTED_TEXT)


key= 5
11000
11101


In [13]:
plain_text=PLAIN_TEXT

In [14]:
cipher_text=encrypted_text


In [15]:
import numpy as np

# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, QuantumRegister, ClassicalRegister#, BasicAer, execute, IBMQ
from qiskit.compiler import assemble
# from qiskit.tools.jupyter import *
from qiskit.visualization import *
# from ibm_quantum_widgets import CircuitComposer
# from qiskit.providers.aer import QasmSimulator, AerSimulator
import numpy as np
# from numpy import pi

# Loading your IBM Quantum account(s)
# provider = IBMQ.load_account()

In [16]:
def load_input(qc, r_key, input_binary, n):
    for i in range(n):
        if (input_binary[i] == "1"):
            qc.x(r_key[i])

In [17]:
def load_key(qc, r_key, n):
    for i in range(n):
        qc.h(r_key[i])

In [18]:
# def cipher(qc, r_key, r_text, n):
def cipher(qc, adder, anc, operand1, operand2):
    qc.append(adder, [anc[0]] + operand1[:] + operand2[:] + [anc[1]])
    # qc.measure(operand2[:] + [anc[1]], cr)
    # tr_circ = transpile(qc, basis_gates = ['u3', 'cx'], optimization_level = 3)
    # result = backend.run(tr_circ).result()
    # counts = result.get_counts()
    # for i in range(n):
    #     qc.cx(r_key[i], r_text[i])

In [19]:
def my_oracle(qc, r_key, r_ancilla, r_cipher, r_text, r_output, n):
    #Compute cipher text possibilities by XORing
    cipher(qc,adder, r_ancilla, r_key, r_text)
    
    #Checking whether the generated cipher text is equal to the given cipher text
    for i in range(n):
        qc.cx(r_cipher[i], r_ancilla[i])
        qc.cx(r_text[i], r_ancilla[i])
        qc.x(r_ancilla[i])
        
    #Set 'output' bit if the cipher text is matched
    qc.mcx(r_ancilla, r_output)
    
    qc.barrier()
    
    #Uncompute cipher to reset ancilla & plain text qubits
    #Reset ancilla qubit
    for i in range(n):
        qc.x(r_ancilla[i])
        qc.cx(r_cipher[i], r_ancilla[i])
        qc.cx(r_text[i], r_ancilla[i])
    qc.barrier()
    #Reset plain text by inverse cipher process
    cipher(qc,adder, r_ancilla, r_key, r_text)
    
    qc.barrier()
        

In [20]:
#The diffuser function is available in Qiskit
def diffuser(nqubits):
    qc = QuantumCircuit(nqubits)
    #Apply transformation |s> -> |00...0> (H gates)
    for qubit in range(nqubits):
        qc.h(qubit)
    #Apply transformation |00...0> -> |11...1> (X gates)
    for qubit in range(nqubits):
        qc.x(qubit)    
    #Do multi-controlled Z gate
    qc.h(nqubits - 1)
    qc.mcx(list(range(nqubits - 1)), nqubits - 1) #multi-controlled Toffoli
    qc.h(nqubits - 1)
    #Apply transformation |11...1> -> |00...0> (X gates)
    for qubit in range(nqubits):
        qc.x(qubit)
    #Apply transformation |00...0> -> |s> (H gates)
    for qubit in range(nqubits):
        qc.h(qubit)
    #We will return the diffuser as a gate
    U_s = qc.to_gate()
    U_s.name = "U$_s$"
    return U_s
    

In [21]:
n = 5
r_text = QuantumRegister(n, 't')
r_key = QuantumRegister(n, 'k')
r_output = QuantumRegister(1, name = 'o')
r_cipher = QuantumRegister(n, 'c')
r_ancilla = QuantumRegister(n+1, 'a')
# r_ancilla = QuantumRegister(2, 'a')

r_class = ClassicalRegister(n)

qc = QuantumCircuit(r_text, r_key, r_cipher, r_ancilla, r_output, r_class)

adder = CDKMRippleCarryAdder(n, kind='full', name='Full Adder')

#loading plain text
# plain_text = '0101'
load_input(qc, r_text, plain_text, n)
print(plain_text)
#Preparing key in a superposition state
load_key(qc, r_key, n)

#loading known cipher text
# cipher_text = '1001'
load_input(qc, r_cipher, cipher_text, n)
print(cipher_text)

# #Preparing output qubit
# qc.x(r_output)
# qc.h(r_output)
# qc.barrier()

# #First Iteration
# my_oracle(qc, r_key, r_ancilla, r_cipher, r_text, r_output, n)
# qc.barrier()
# qc.append(diffuser(n), r_key)
# qc.barrier()

# #Second Iteration
# my_oracle(qc, r_key, r_ancilla, r_cipher, r_text, r_output, n)
# qc.barrier()
# qc.append(diffuser(n), r_key)
# qc.barrier()

# qc.measure(r_key, r_class)
# #qc.draw()

from math import pi, sqrt
num_iterations = int(pi/4 * 2**(n/2.0)+1)

# Perform the remaining iterations
for i in range(num_iterations):
    my_oracle(qc, r_key, r_ancilla, r_cipher, r_text, r_output, n)
    qc.barrier()
    qc.append(diffuser(n), r_key)
    qc.barrier()
    
qc.measure(r_key, r_class)
# qc.draw()

11000
11101


In [22]:
# from qiskit_aer import AerSimulator
# aer_sim = AerSimulator()
# pm = generate_preset_pass_manager(backend=aer_sim, optimization_level=1)
# sim = pm.run(qc)

# sim = AerSimulator(method = 'matrix_product_state')
# sim = Aer.get_backend('qasm_simulator')

from qiskit.providers.basic_provider import BasicSimulator
sim = BasicSimulator()


shots = 1024
#Run and get the counts, using the matrix_product_state method
tcirc = transpile(qc, sim)
result = sim.run(tcirc, shots=1024).result()
print (result.get_counts())
counts = result.get_counts() #counts = result.get_counts(0)
b, c = max(counts.items(), key=lambda x: x[1])
print("Key is :", b[::-1], "with probability =", c/shots)

{'00010': 35, '01000': 23, '11000': 29, '10100': 28, '01010': 33, '10110': 32, '11001': 29, '00001': 32, '10111': 42, '01100': 32, '10011': 44, '00100': 23, '01001': 31, '00101': 37, '00111': 37, '11011': 33, '00011': 28, '00110': 22, '10001': 38, '11010': 29, '11101': 45, '11100': 35, '01111': 22, '01110': 23, '01101': 40, '10000': 38, '01011': 39, '11110': 30, '11111': 39, '10101': 28, '10010': 22, '00000': 26}
Key is : 10111 with probability = 0.0439453125
